In [ ]:
#!pip install torch torchvision 

In [32]:
# Imports - check to make sure all are installed 
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import os
import pandas as pd 
import numpy as np 
from PIL import Image

In [33]:
# Access files
# Images and labels stored in different folders

df = pd.read_csv("All_TIF_labeled_tiles.csv")
images = "./CNN_pngs"

df['filepath'] = df.apply(
    lambda row: os.path.join(images, str(row['File'])),
    axis=1
)

In [ ]:
# CHECK - make sure it's working
# Print to see
missing = df[~df['filepath'].apply(os.path.exists)]
if len(missing) > 0:
    print(f"Warning: {len(missing)} files missing!")
    #print(missing[['File', 'filepath']].head())

In [ ]:
# Check imgs in df 
n_rows = 3
n_cols = 3

f, axarr = plt.subplots(n_rows, n_cols)

for row in range(n_rows):
  for col in range(n_cols):
    image = Image.open(df.sample(n=1)["filepath"].iloc[0]).convert(("RGB"))
    axarr[row, col].imshow(image)
    axarr[row, col].axis("off")

plt.show()

In [34]:
# Encode labels
# 1. The R/C class could be encoded several ways
# 2. For Azimuth, assign degrees or directions (think about how we will limit this later)

# Residential/Commercial class
# Option 1-binary
df['R/C'] = df['R/C'].astype(str).map({'R': 0, 'C': 1})

# Azimuth class
# Technically in degrees, but labeled N, E, S, W, NE, SE, SW, NW. Convert to int
azimuth_map = {
    'N': 0, 'NE': 1, 'E': 2, 'SE': 3,
    'S': 4, 'SW': 5, 'W': 6, 'NW': 7,
    '0': 8, 0: 8
}

df['Azimuth'] = df['Azimuth'].astype(str).map(azimuth_map)



In [35]:
print("R/C distribution:", df['R/C'].value_counts())
print("Azimuth distribution:", df['Azimuth'].value_counts())

R/C distribution: R/C
1.0    193
0.0    187
Name: count, dtype: int64
Azimuth distribution: Azimuth
8    528
4     13
6      5
5      4
2      2
Name: count, dtype: int64


In [36]:
# Split train/val/test sets
SEED = 42
train = df.sample(frac=0.7, random_state=SEED)

test = df.drop(train.index)
val = test.sample(frac=0.5)
test = test.drop(val.index)

# Will need to handle class imbalance later
print(train.shape)
print(test.shape)
print(val.shape)

(386, 8)
(83, 8)
(83, 8)


In [ ]:
# Image handling:
# Need to check size, normalization, and possibly transform teh images 
sample_img = Image.open(df.iloc[0]['filepath'])
print(f"Image size: {sample_img.size}")

This is the size we set for our tiling of the TIF images. If possible, let's keep this scale and see the model progress before reducing size.

In [37]:
# Use PyTorch Dataset to build custom dataset for processing
class RoofDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        try:
            row = self.dataframe.iloc[idx]
            image = Image.open(row['filepath']).convert("RGB")
            if self.transform:
                image = self.transform(image)
        
        except Exception as e:
            print(f"Error loading image at index {idx}: {e}")
        
        #labels
        num_roofs  = torch.tensor(row['Num_roofs'], dtype=torch.float32)
        has_roofs  = torch.tensor(row['Has_roofs'], dtype=torch.long)
        rc_class   = torch.tensor(row['R/C'], dtype=torch.long)
        pv_class   = torch.tensor(row['PV'], dtype=torch.long)
        azimuth    = torch.tensor(row['Azimuth'], dtype=torch.long)
        if pd.isna(row['Azimuth']):
            raise ValueError(f"NaN azimuth at index {idx}")
        
        return image, {
            'num_roofs': num_roofs,
            'has_roofs': has_roofs,
            'rc_class': rc_class,
            'pv_class': pv_class,
            'azimuth': azimuth
        }


We want all labels to be predicted from the CNN, which will require a multi-class model. WHat kind of predictions are we looking for with each label?

Num_roofs: int (regression?)

Has_roofs: binary, classification

R/C: binary(?), classification

PV: binary, classification

Azimuth: int

Model goal:
regression + classification 

In [38]:
# Transform datasets 
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float32)
])

# If rezizing is needed, add here
# transform = transforms.Compose([
#     transforms.Resize((128, 128))

# If normalization is needed, add here
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

In [39]:
train_dataset = RoofDataset(train, transform)
test_dataset = RoofDataset(test, transform)
val_dataset = RoofDataset(val, transform)

In [40]:
# CHECK - make sure it's working
train_dataset.__getitem__(2)

(tensor([[[0.3176, 0.3373, 0.3490,  ..., 0.5412, 0.5451, 0.5490],
          [0.3098, 0.3373, 0.3412,  ..., 0.5412, 0.5412, 0.5451],
          [0.2941, 0.3294, 0.3294,  ..., 0.5373, 0.5451, 0.5451],
          ...,
          [0.2941, 0.2941, 0.2863,  ..., 0.5804, 0.5804, 0.5569],
          [0.2980, 0.2941, 0.2980,  ..., 0.5882, 0.5882, 0.5765],
          [0.2980, 0.2902, 0.3020,  ..., 0.5882, 0.6078, 0.6039]],
 
         [[0.3020, 0.3294, 0.3412,  ..., 0.5294, 0.5333, 0.5373],
          [0.2980, 0.3412, 0.3333,  ..., 0.5216, 0.5333, 0.5373],
          [0.2824, 0.3333, 0.3216,  ..., 0.5216, 0.5294, 0.5333],
          ...,
          [0.3059, 0.3059, 0.2980,  ..., 0.5686, 0.5725, 0.5451],
          [0.3098, 0.3059, 0.3020,  ..., 0.5725, 0.5725, 0.5647],
          [0.3137, 0.3020, 0.3020,  ..., 0.5765, 0.5843, 0.5922]],
 
         [[0.3020, 0.3294, 0.3412,  ..., 0.4941, 0.5020, 0.5098],
          [0.2941, 0.3255, 0.3294,  ..., 0.4941, 0.5020, 0.5098],
          [0.2824, 0.3176, 0.3176,  ...,

In [41]:
# Specify the parameters
# small and basic at first, adjust later
lr = 0.001
batch_size = 32
epochs = 20


In [42]:
# Create dataloader for the custom dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) 
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Build the model
class RoofCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # backbone
        self.base = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        in_feats = self.base.fc.in_features
        self.base.fc = nn.Identity()
        
        # heads
        self.num_roofs = nn.Linear(in_feats, 1)      
        self.has_roofs = nn.Linear(in_feats, 2)      
        self.rc_class  = nn.Linear(in_feats, 3)      
        self.pv_class  = nn.Linear(in_feats, 2)      
        self.azimuth   = nn.Linear(in_feats, 9)      
    
    def forward(self, x):
        feats = self.base(x)
        
        return {
            "num_roofs": self.num_roofs(feats),
            "has_roofs": self.has_roofs(feats),
            "rc_class": self.rc_class(feats),
            "pv_class": self.pv_class(feats),
            "azimuth": self.azimuth(feats)
        }

In [ ]:
# Specify the loss type
# since there are different types of outputs, we need different loss functions
criterion_mse = nn.MSELoss()
criterion_ce = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)

model = RoofCNN().to(device)

In [ ]:
# Loss weights (tune these based on importance/results)
loss_weights = {
    'num_roofs': 1.0,
    'has_roofs': 1.0,
    'rc_class': 1.0,
    'pv_class': 1.0,
    'azimuth': 1.0
}

In [ ]:
# Collect training history for plotting later
history = {
    'train_loss': [], 'val_loss': [],
    'train_acc_has': [], 'val_acc_has': [],
    'train_acc_rc': [], 'val_acc_rc': [],
    'train_acc_pv': [], 'val_acc_pv': [],
    'train_acc_az': [], 'val_acc_az': []
}

In [ ]:
# Build the training and testing loops
for epoch in range(epochs):
    model.train()
    

In [ ]:
# Verify results

In [ ]:
#Plot the results